<a href="https://colab.research.google.com/github/DeepInsider/playground-data/blob/master/docs/articles/tf2_keras_neuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 「ニューラルネットワーク ― 仕組みの理解×初めての実装（全3回）」 ― 連載『TensorFlow 2.0＋Keras（ tf.keras）入門』のノートブック

<table valign="middle">
  <td>
    <a target="_blank" href="https://deepinsider.jp/tutor/deeplearningdat"> <img src="https://re.deepinsider.jp/img/ml-logo/manabu.svg"/>Deep Insiderで記事を読む</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/DeepInsider/playground-data/blob/master/docs/articles/deeplearningdat.ipynb"> <img src="https://re.deepinsider.jp/img/ml-logo/gcolab.svg" />Google Colabで実行する</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/DeepInsider/playground-data/blob/master/docs/articles/deeplearningdat.ipynb"> <img src="https://re.deepinsider.jp/img/ml-logo/github.svg" />GitHubでソースコードを見る</a>
  </td>
</table>

# 第1回　初めてのニューラルネットワーク実装の準備をしよう ― 仕組み理解×初実装（前編）

## ■ディープラーニングの大まかな流れ

- （1）データ準備
- （2）問題種別
- （3）前処理
- （4）“手法” の選択： モデルの定義
- （5）“学習方法” の設計： モデルの生成
- （6）学習 ： トレーニング
- （7）評価
- （8）テスト

## ■（1）データ準備

### 【チェック】Pythonバージョン（※3系を使うこと）
Colabにインストール済みのものを使う。もし2系になっている場合は、メニューバーの［ランタイム］－［ランタイムのタイプを変更］をクリックして切り替えてほしい。

In [0]:
import sys
print('Python', sys.version)
# Python 3.6.8 (default, Jan 14 2019, 11:02:34) …… などと表示される

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]


### リスト1　座標点データを生成するライブラリのインストール

In [0]:
!pip install playground-data

  Created wheel for playground-data: filename=playground_data-1.0.1-py2.py3-none-any.whl size=20362 sha256=22803dfd47caeb083e794a7f1afd3eb8c295829c3a57da6d8901c31a2b0bbf74
  Stored in directory: /root/.cache/pip/wheels/33/3f/7e/477ea4684f873e5ba9f762bc7b91d68f30e852f16866e98f13
Successfully built playground-data


## ■（2）問題種別

### リスト2　問題種別とデータ種別の選択

In [0]:
# playground-dataライブラリのplygdataパッケージを「pg」という別名でインポート
import plygdata as pg

# 問題種別で「分類（Classification）」を選択し、
# データ種別で「2つのガウシアンデータ（TwoGaussData）」を選択する場合の、
# 設定値を定数として定義
PROBLEM_DATA_TYPE = pg.DatasetType.ClassifyTwoGaussData

# ※実際のデータ生成は、後述の「pg.generate_data()関数の呼び出し」で行う

## ■（3）前処理

### リスト3　前処理としてのデータ分割

In [0]:
# 各種設定を定数として定義
VALIDATION_DATA_RATIO = 0.5  # データの何％を訓練【Training】用に？ (残りは精度検証【Validation】用) ： 50％
DATA_NOISE = 0.0             # ノイズ： 0％

# 定義済みの定数を引数に指定して、データを生成する
data_list = pg.generate_data(PROBLEM_DATA_TYPE, DATA_NOISE)

# データを「訓練用」と「精度検証用」を指定の比率で分割し、さらにそれぞれを「データ（X）」と「教師ラベル（y）」に分ける
X_train, y_train, X_valid, y_valid = pg.split_data(data_list, validation_size=VALIDATION_DATA_RATIO)

### 【チェック】データ分割後の各変数の内容例

In [0]:
# それぞれ5件ずつ出力
print('X_train:'); print(X_train[:5])
print('y_train:'); print(y_train[:5])
print('X_valid:'); print(X_valid[:5])
print('y_valid:'); print(y_valid[:5])

X_train:
[[ 1.32246294  2.41059059]
 [-2.99999629 -2.94425406]
 [-2.10387631 -2.91059905]
 [ 1.88077669  1.65356138]
 [ 2.06380792  1.35886889]]
y_train:
[[ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]]
X_valid:
[[ 2.69323277  1.37843374]
 [-1.26132847 -4.0659741 ]
 [ 2.7252259   2.22092912]
 [ 2.0749284   3.05473508]
 [-1.49418439 -1.50701399]]
y_valid:
[[ 1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]]


# 第2回　ニューラルネットワーク最速入門 ― 仕組み理解×初実装（中編）

# 第3回　ディープラーニング最速入門 ― 仕組み理解×初実装（後編）

# 本ノートブックは未完成です。連載の進行に合わせて完成します。<!--お疲れさまでした。『ニューラルネットワーク、仕組みの理解×初めての実装』は修了です。-->